### ORI Final Analysis
### 02. National Secutiry

#### 10/24/2024
#### This script processes data and creates the natioanl security submodel. 

In [3]:
#import system modules and set environment
import arcpy
import os
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI'

#define spatial reference (EPSG for NAD 83 UTM Zone 11N)
spatial_ref = arcpy.SpatialReference(26911)

#create a copy of the ORI area hex grid
#hex_grid = 'data/ORI_Area_of_Interest_grid.shp'
hex_grid = 'data/input_data/constrained.shp'
ns_submodel = 'data/input_data/national_security_submodel'
arcpy.management.CopyFeatures(hex_grid, ns_submodel)

#define area of interest
aoi = 'data/ORI_Area_of_Interest.shp'

#create geopackage
ns_gpkg_path = 'data/input_data/ORI_National_Security.gpkg'
if not os.path.exists(ns_gpkg_path):
    arcpy.management.CreateSQLiteDatabase(ns_gpkg_path, "GEOPACKAGE")

In [4]:
# Military Training Routes (ori_ns_01)

#define file
file_path = 'data/source_data/MTR_Segment.shp'

#create feature layer for the file
arcpy.management.MakeFeatureLayer(file_path, "mtr_intersect")

#select file where it itersects the grid
arcpy.management.SelectLayerByLocation(
    in_layer= "mtr_intersect",
    overlap_type="INTERSECT",
    select_features= ns_submodel,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project("mtr_intersect", file_project, spatial_ref)

##buffer by route width
left_buffer = "in_memory/left_buffer"
right_buffer = "in_memory/right_buffer"

#create new fields
if not arcpy.ListFields(file_project, "widthleft_m"):
    arcpy.AddField_management(file_project, "widthleft_m", "DOUBLE")
if not arcpy.ListFields(file_project, "widthright_m"):
    arcpy.AddField_management(file_project, "widthright_m", "DOUBLE")

#calculate width in meters
arcpy.CalculateField_management(file_project, "widthleft_m", "!widthleft! * 1609.344", "PYTHON3")
arcpy.CalculateField_management(file_project, "widthright_m", "!widthright! * 1609.344", "PYTHON3")

#create buffers
arcpy.Buffer_analysis(file_project, left_buffer, "widthleft_m", line_side="LEFT", dissolve_option="NONE")
arcpy.Buffer_analysis(file_project, right_buffer, "widthright_m", line_side="RIGHT", dissolve_option="NONE")

#merge and dissolve buffers
merged_buffers = "in_memory/merged_buffers"
arcpy.Merge_management([left_buffer, right_buffer], merged_buffers)

dissolved_buffer = "in_memory/dissolved_buffer"
arcpy.Dissolve_management(merged_buffers, dissolved_buffer)

#save to geopackage
output_name = 'MTR_buffer'
arcpy.conversion.FeatureClassToFeatureClass(dissolved_buffer, ns_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(ns_submodel,
                                                                     'INTERSECT',
                                                                     dissolved_buffer)
arcpy.management.CalculateField(file_selection, 'ori_ns_01', '0.5', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(ns_submodel, 
                                                        'INTERSECT', 
                                                        dissolved_buffer, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_ns_01', '1', field_type="DOUBLE")


<Result 'national_security_submodel_L2'>

In [5]:
# Formerly Used Defense Sites (ori_ns_02)

#define file
gpkg_path = 'data/source_data/FormerlyUsedDefenseSite.gpkg'
file_path = f"{gpkg_path}/main.FormerlyUsedDefenseSite"

#create feature layer for the file
arcpy.management.MakeFeatureLayer(file_path, "fuds_intersect")

#select file where it itersects the grid
arcpy.management.SelectLayerByLocation(
    in_layer= "fuds_intersect",
    overlap_type="INTERSECT",
    select_features= ns_submodel,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project("fuds_intersect", file_project, spatial_ref)

#save to geopackage
output_name = 'FUDS'
arcpy.conversion.FeatureClassToFeatureClass(file_project, ns_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(ns_submodel,
                                                                     'INTERSECT',
                                                                     file_project)
arcpy.management.CalculateField(file_selection, 'ori_ns_02', '0.5', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(ns_submodel, 
                                                        'INTERSECT', 
                                                        file_project, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_ns_02', '1', field_type="DOUBLE")

<Result 'national_security_submodel_L4'>

In [6]:
# National Security Submodel

#call shapefile
ns_submodel = 'data/input_data/national_security_submodel.shp'

#create a new field for national security
ns_field = "ns_final"

existing_fields = [f.name for f in arcpy.ListFields(ns_submodel)]

if ns_field not in existing_fields:
    arcpy.management.AddField(ns_submodel, ns_field, 'DOUBLE')
    
#expression to get final constraints
final_expression = '(!ori_ns_01! * !ori_ns_02!) ** (1/2)'

# Calculate the geometric mean and store the result in the 'final_ns' attribute
arcpy.CalculateField_management(ns_submodel, ns_field, final_expression, 'PYTHON3')


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/national_security_submodel.shp'>